In [6]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx

In [7]:
text = "The scenes at the final whistle said it all. An explosion of noise from most of those in attendance while, in front of them, those in claret and blue celebrated and sighed in relief in equal measure. It had been that sort of game for West Ham – tough, challenging, nerve-racking but ultimately glorious. Little wonder David Moyes took his time departing down the tunnel, the Scot milking the applause and rightly so, for this is a manager reborn in charge of a team that continues to soar. Indeed, this was not only West Ham’s first league victory over Liverpool since January 2016 but one that saw them leapfrog their opponents into third. They are also level on points with Manchester City and only three behind Chelsea and, while the idea of Moyes’s men qualifying for the Champions League and even challenging for the title may seem far-fetched, it simply cannot be ruled out given their form and performances. This is their fourth league win in a row and each, in its own way, has been deserved. Here they were forced to show great togetherness and resolve against opponents who dominated possession and territory and carried a goal threat right to the last. The hosts were forced on to the back foot but, as has been typical of them for some time now, they did not wilt while, going forward, they yet again showed craft and guile, unsettling Liverpool on the counterattack and twice piercing them from what has become a formidable weapon: the set piece. For Liverpool it was a chastening afternoon, halting their unbeaten run at 25 games and bringing into sharp focus the defensive frailties that threaten their hopes of winning prizes at home and abroad. This was the sixth occasion this season that they have conceded two or more goals and, while that resulted from a collective slackness, there was a standout figure: Alisson. The Brazilian, normally so secure and reliable, was a central presence for all three West Ham goals, most strikingly the first, given it was an own goal on his part. Only four minutes had been played when Pablo Fornals swung a corner into the six-yard area and Liverpool’s goalkeeper made a mess of clearing it with his left arm, the ball grazing off his thumb and drifting into the net. Liverpool’s players protested to the referee, Craig Pawson, insisting Alisson had been fouled by Angelo Ogbonna as the pair jumped together. There came a VAR check and it was decided there had been no foul on Ogbonna’s part. Not that Jürgen Klopp saw it that way. He was immediately incensed by the decision and reiterated that stance afterwards, insisting Ogbonna “hits Alisson’s arm” as they rose and therefore prevented him from making clean contact with the ball. And that was not the end of the German’s ire in regard to the officiating. Five minutes later he was left raging with Pawson for not sending off Aaron Cresswell for his challenge on Jordan Henderson, with the West Ham left-back catching the Liverpool captain on the knee of his standing leg as he maneuvered down the right flank. It definitely could have led to a red yet, incredibly, after another VAR check, not even a foul was given. A sense of injustice permeated through the visitors’ ranks and they channelled it by taking full control of proceedings. After 41 minutes they got the equaliser their efforts deserved via a wonderfully executed free-kick from Trent Alexander-Arnold from the edge of the area, the ball curling over the wall and into the top right-hand corner of the net. It was a blow for West Ham so close to half-time, especially given they had also lost Ogbonna after 22 minutes due to a cut to the centre-back’s face resulting from being accidentally caught by Diojo Jota’s elbow. But they did not feel sorry for themselves and, come the second half, found not only a second wind but further gears. The hosts continued to sit deep but now they were pouring forward on the break, in the process establishing the platform for a barnstorming victory against increasingly lacklustre opponents. Jarrod Bowen and Fornals stood out and it was that pair who combined for West Ham’s second goal on 67 minutes, with Bowen charging forward after possession had been overturned in midfield and playing in Fornals, whose left-footed shot too easily went through the grasp of Alisson. Cue raucous cheers among the home fans, with the volume turned up even further when Kurt Zouma made it 3-1 on 74 minutes with a far-post header from Bowen’s right-sided corner. Alisson scampered across in an attempt to block the ball but got nowhere near it. For West Ham it was also a sixth set-piece goal in six games. Liverpool came forward with renewed vigour and scored again on 83 minutes after Divock Origi, on as a substitute, sent a low drive past Lukasz Fabianski. A comeback suddenly felt on and looked to have been completed when Sadio Mané met Alexander-Arnold’s cross with a diving header that sent the ball past Fabianski and seemingly into the far corner of the net. It went narrowly wide and, soon after, West Ham had secured a triumph that sent this venue into raptures."
text[0:2000]

'The scenes at the final whistle said it all. An explosion of noise from most of those in attendance while, in front of them, those in claret and blue celebrated and sighed in relief in equal measure. It had been that sort of game for West Ham – tough, challenging, nerve-racking but ultimately glorious. Little wonder David Moyes took his time departing down the tunnel, the Scot milking the applause and rightly so, for this is a manager reborn in charge of a team that continues to soar. Indeed, this was not only West Ham’s first league victory over Liverpool since January 2016 but one that saw them leapfrog their opponents into third. They are also level on points with Manchester City and only three behind Chelsea and, while the idea of Moyes’s men qualifying for the Champions League and even challenging for the title may seem far-fetched, it simply cannot be ruled out given their form and performances. This is their fourth league win in a row and each, in its own way, has been deserved

In [8]:
sentences = text.split(". ")

In [10]:
sentences[0:9] #Sample

['The scenes at the final whistle said it all',
 'An explosion of noise from most of those in attendance while, in front of them, those in claret and blue celebrated and sighed in relief in equal measure',
 'It had been that sort of game for West Ham – tough, challenging, nerve-racking but ultimately glorious',
 'Little wonder David Moyes took his time departing down the tunnel, the Scot milking the applause and rightly so, for this is a manager reborn in charge of a team that continues to soar',
 'Indeed, this was not only West Ham’s first league victory over Liverpool since January 2016 but one that saw them leapfrog their opponents into third',
 'They are also level on points with Manchester City and only three behind Chelsea and, while the idea of Moyes’s men qualifying for the Champions League and even challenging for the title may seem far-fetched, it simply cannot be ruled out given their form and performances',
 'This is their fourth league win in a row and each, in its own way

In [12]:
sentence_tokens = [s.lower().split(" ") for s in sentences]
sentence_tokens[0:2] # sample

[['the', 'scenes', 'at', 'the', 'final', 'whistle', 'said', 'it', 'all'],
 ['an',
  'explosion',
  'of',
  'noise',
  'from',
  'most',
  'of',
  'those',
  'in',
  'attendance',
  'while,',
  'in',
  'front',
  'of',
  'them,',
  'those',
  'in',
  'claret',
  'and',
  'blue',
  'celebrated',
  'and',
  'sighed',
  'in',
  'relief',
  'in',
  'equal',
  'measure']]

## Sentence similarity

In [6]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [7]:
sentence_similarity(sentence_tokens[5], sentence_tokens[10])

0.12734290799340264

In [8]:
sentence_tokens[5]

['they',
 'are',
 'also',
 'level',
 'on',
 'points',
 'with',
 'manchester',
 'city',
 'and',
 'only',
 'three',
 'behind',
 'chelsea',
 'and,',
 'while',
 'the',
 'idea',
 'of',
 'moyes’s',
 'men',
 'qualifying',
 'for',
 'the',
 'champions',
 'league',
 'and',
 'even',
 'challenging',
 'for',
 'the',
 'title',
 'may',
 'seem',
 'far-fetched,',
 'it',
 'simply',
 'cannot',
 'be',
 'ruled',
 'out',
 'given',
 'their',
 'form',
 'and',
 'performances']

In [9]:
sentence_tokens[10]

['this',
 'was',
 'the',
 'sixth',
 'occasion',
 'this',
 'season',
 'that',
 'they',
 'have',
 'conceded',
 'two',
 'or',
 'more',
 'goals',
 'and,',
 'while',
 'that',
 'resulted',
 'from',
 'a',
 'collective',
 'slackness,',
 'there',
 'was',
 'a',
 'standout',
 'figure:',
 'alisson']

In [10]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

In [11]:
stop_words = stopwords.words('english')

In [12]:
sentence_sim = build_similarity_matrix(sentence_tokens, stop_words)
sentence_sim

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.1754116 ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.16269784,
        0.06726728],
       [0.        , 0.        , 0.        , ..., 0.16269784, 0.        ,
        0.06201737],
       [0.        , 0.        , 0.1754116 , ..., 0.06726728, 0.06201737,
        0.        ]])

## Rank sentences in similarity based on PageRank

In [15]:
sentence_similarity_graph = nx.from_numpy_array(sentence_sim)
scores = nx.pagerank(sentence_similarity_graph)

In [16]:
sentence_similarity_graph[5]

AtlasView({4: {'weight': 0.04969039949999532}, 6: {'weight': 0.07273929674533075}, 10: {'weight': 0.05337605126836242}, 11: {'weight': 0.09938079899999064}, 19: {'weight': 0.11111111111111116}, 22: {'weight': 0.0820609939862218}, 23: {'weight': 0.05555555555555558}, 26: {'weight': 0.043033148291193535}, 28: {'weight': 0.06804138174397711}, 31: {'weight': 0.05337605126836242}})

## Rank the top sentence

In [17]:
ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
print("Top ranked_sentence order are ", ranked_sentence)

Top ranked_sentence order are  [(0.06790640075907707, 'For West Ham it was also a sixth set-piece goal in six games'), (0.06128902405688086, 'Indeed, this was not only West Ham’s first league victory over Liverpool since January 2016 but one that saw them leapfrog their opponents into third'), (0.05723884335481713, 'The Brazilian, normally so secure and reliable, was a central presence for all three West Ham goals, most strikingly the first, given it was an own goal on his part'), (0.055654899308902245, 'It was a blow for West Ham so close to half-time, especially given they had also lost Ogbonna after 22 minutes due to a cut to the centre-back’s face resulting from being accidentally caught by Diojo Jota’s elbow'), (0.053584058232932386, 'It went narrowly wide and, soon after, West Ham had secured a triumph that sent this venue into raptures.'), (0.052424084576485674, 'Five minutes later he was left raging with Pawson for not sending off Aaron Cresswell for his challenge on Jordan Hen

## Pick top n

In [18]:
top_n = 2
summarize_text = []
for i in range(top_n):
      summarize_text.append("".join(ranked_sentence[i][1]))

print("Summarize Text: \n", ". ".join(summarize_text))

Summarize Text: 
 For West Ham it was also a sixth set-piece goal in six games. Indeed, this was not only West Ham’s first league victory over Liverpool since January 2016 but one that saw them leapfrog their opponents into third
